<h1 style='background-color: #0B3861; padding: 10px; color: white; text-align: center;'> RIGHTMOVE </h1>


<h1 style='background-color: #0B3861; padding: 10px; color: white'> Sales </h1>


First things first, we will import the required libraries

In [46]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd


Next, we will load in the dataset containing the list of postcode for this project

In [47]:
codes = pd.read_csv('London postcode districts.xlsx - PC DIST.csv')

Now, we will create a function called 'rightMove_sales' to scrap properties that are for sale on the Rightmove website

In [48]:
def rightMove_sales(postcodes, Trans_type, website, df):
    """
    This function scrapes property data from Rightmove for a given list of postcodes and transaction type,
    and returns the data as a pandas DataFrame.

    Arguments:
    postcodes -- a list of postcodes for which to scrape property data
    Trans_type -- the transaction type of the properties to be scraped ('sales' or 'rent')
    website -- the name of the website being scraped (in this case, 'Rightmove')
    df -- an empty pandas DataFrame to store the scraped data

    Returns:
    df1 -- a pandas DataFrame containing the scraped property data
    """
    
    # Set the path to the ChromeDriver executable
    s = Service("C:\\Users\\user\\Downloads\\Set ups\\chromedriver_win32\\chromedriver.exe")
    
    # Launch the ChromeDriver with the specified service
    driver = webdriver.Chrome(service= s)
    
    # Navigate to the specified URL and maximize the window
    driver.get('https://www.rightmove.co.uk/')
    driver.maximize_window()
    
    # Wait for 3 seconds to allow the page to load
    time.sleep(3)

    # Find the search bar and enter the search query
    time.sleep(2)
    search = driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/main/div[1]/div/div/div/div/input')
    search.send_keys(postcode) 
    time.sleep(2)
    
    #Click the find search button
    driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/main/div[1]/div/div/div/button[1]').click()
    time.sleep(2)
    
    #Click the find properties button
    driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div/div[1]/div/form/fieldset[2]/div[4]/button').click()
    
    # Initialize empty lists for storing scraped data
    Trans_type = []
    address = []
    types = []
    bedrooms = []
    bathrooms = []
    prices = []
    desc = []
    date_added = []
    agent_list = []
    property_url = []
    website = []
    
    # Create an empty dataframe to store the scraped data
    df = pd.DataFrame()
    
    # Setting the page number to be 1 and looping over the available pages for each postcode
    i = 1
    while True:
        
        # Scrapping data for the required features
        time.sleep(2)
        print("{} {} {} {}".format('scraping page', i,'from', postcode ))
        address_list = driver.find_elements(by='class name', value='propertyCard-address')
        type_list = driver.find_elements(By.XPATH, "//div/div/div[4]/div[1]/div[2]/a/div/span[1]")
        bedroom_list = driver.find_elements(By.XPATH, '//div/div/div[4]/div[1]/div[2]/a/div/span[3]')
        bathroom_list = driver.find_elements(By.XPATH, '//div/div/div[4]/div[1]/div[2]/a/div/span[5]')
        price_list = driver.find_elements(By.XPATH, '//div/div/div[3]/div/a/div[1]')
        desc_list = driver.find_elements(By.XPATH, '//div/div/div[4]/div[1]/div[3]/a/span/span')
        date_added_list = driver.find_elements(By.XPATH, '//div/div/div[4]/div[2]/div/span')
        agent_list_list = driver.find_elements(By.XPATH, '//div/div/div[4]/div[2]/div')
        property_url_list = driver.current_url
        Trans_type_list = Trans_type
        website_list = website
        for address_item, type_item, bedroom_item, bathroom_item, price_item, desc_item, date_added_item, agent_list_item in zip(address_list, type_list, bedroom_list, bathroom_list, price_list, desc_list, date_added_list, agent_list_list):
            address.append(address_item.text)
            types.append(type_item.text)
            bedrooms.append(bedroom_item.text)
            bathrooms.append(bathroom_item.text)
            prices.append(price_item.text)
            desc.append(desc_item.text)
            date_added.append(date_added_item.text)
            agent_list.append(agent_list_item.text)
            property_url.append(property_url_list)
            Trans_type.append(Trans_type_list)
            website.append(website_list)
            
        # Find the next button   
        next_botton = driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[3]/div/div/div/div[3]/button/span')
        
        # Find the last next button
        last_botton_check = driver.find_element(By.XPATH, '/html/body/div[3]/div[2]/div[1]/div[3]/div/div/div/div[3]/button')
        
        # Checks for the last next button and breaks off
        html = last_botton_check.get_attribute('outerHTML') 
        if 'disabled' in html:
            print('------------------------------- SCRAPING COMPLETED FOR ' + postcode)
            break
            
        #Click the next button if it's not the last    
        next_botton.click()
        
        #Increment the page number
        i += 1
        time.sleep(2)
        
    # Create a dataframe to store data scrapped for each postcode
    df1 = pd.DataFrame({'Location': postcode, 'Tansaction_Type': 'Sales','Property_Type':types, 'Address' :address, 'Bedrooms': bedrooms, 'Bathrooms':bathrooms, 'Price':prices, 'Description': desc, 'Listing_Date':date_added, 'Agent':agent_list, 'Listing_Source': 'RightMove', 'listing_URL':property_url})
    df1.insert(0, 'Unique_Id', [f'{postcode}S{i+1:05d}' for i in range(len(df1))])
    print('Total numbers of properties available in ' + postcode + ' is ' + str(df1.shape[0]))

    # Concat the dataframe obtain for all postcodes
    df = pd.concat([df, df1], ignore_index=True)
    
    # Return a dataframe
    return df1



Now, the data for properties available for sale at Rightmove is scrapped

In [36]:
# create an empty DataFrame outside the function
df = pd.DataFrame()

# loop through availabe postcodes and web scrap
for postcode in codes['Postcode district']:
    
    # call the function and pass the empty DataFrame as an argument
    df1 = rightMove_sales(postcode, 'Sales', 'RightMove', df)
    
    # append the df1 DataFrame to the empty DataFrame
    df = pd.concat([df, df1], ignore_index=True)
print('------------------------ WEB SCRAPING COMPLETED: OVERALL TOTAL NUMBER OF PROPERTIES SCRAPED IS ' + str(df.shape[0]))



scraping page 1 from BR1
scraping page 2 from BR1
scraping page 3 from BR1
scraping page 4 from BR1
scraping page 5 from BR1
scraping page 6 from BR1
scraping page 7 from BR1
scraping page 8 from BR1
scraping page 9 from BR1
scraping page 10 from BR1
scraping page 11 from BR1
scraping page 12 from BR1
scraping page 13 from BR1
scraping page 14 from BR1
scraping page 15 from BR1
------------------------------- SCRAPING COMPLETED FOR BR1
Total numbers of properties available in BR1 is 351
scraping page 1 from BR2
scraping page 2 from BR2
scraping page 3 from BR2
scraping page 4 from BR2
scraping page 5 from BR2
scraping page 6 from BR2
scraping page 7 from BR2
scraping page 8 from BR2
scraping page 9 from BR2
scraping page 10 from BR2
scraping page 11 from BR2
scraping page 12 from BR2
------------------------------- SCRAPING COMPLETED FOR BR2
Total numbers of properties available in BR2 is 282
scraping page 1 from BR3
scraping page 2 from BR3
scraping page 3 from BR3
scraping page 4 fro

KeyboardInterrupt: 

In case the web scraping breaks while running due to network, we can continue the scrapping process from the last postcode as follows

In [49]:
# create an empty DataFrame outside the function
df = pd.DataFrame()

# loop through postcodes
for postcode in codes.loc[264:]['Postcode district']:
    
    # call the function and pass the empty DataFrame as an argument
    df1 = rightMove_sales(postcode, 'Sales', 'RightMove', df)
    
    # append the df1 DataFrame to the empty DataFrame
    df = pd.concat([df, df1], ignore_index=True)
print('------------------------ WEB SCRAPING COMPLETED: OVERALL TOTAL NUMBER OF PROPERTIES SCRAPED IS ' + str(df.shape[0]))



scraping page 1 from WC2E
------------------------------- SCRAPING COMPLETED FOR WC2E
Total numbers of properties available in WC2E is 19
scraping page 1 from WC2H
------------------------------- SCRAPING COMPLETED FOR WC2H
Total numbers of properties available in WC2H is 19
scraping page 1 from WC2N
------------------------------- SCRAPING COMPLETED FOR WC2N
Total numbers of properties available in WC2N is 18
scraping page 1 from WC2R
scraping page 2 from WC2R
scraping page 3 from WC2R
------------------------------- SCRAPING COMPLETED FOR WC2R
Total numbers of properties available in WC2R is 51
------------------------ WEB SCRAPING COMPLETED: OVERALL TOTAL NUMBER OF PROPERTIES SCRAPED IS 107


Saving the dataframe into a csv file

In [ ]:
#df.to_csv('RightMove_Sales_data.csv')

Loading the dataset and removing an unwanted column

In [24]:
df_RM_sales = pd.read_csv('RightMove_Sales_data.csv')
df_RM_sales = df_RM_sales.drop('Unnamed: 0', axis=1)

In [25]:
# Check the shape of the dataset
df_RM_sales.shape

(49641, 13)

Almost 50,000 properties were scrapped

In [26]:
df_RM_sales

,Unique_Id,Location,Tansaction_Type,Property_Type,Address,Bedrooms,Bathrooms,Price,Description,Listing_Date,Agent,Listing_Source,listing_URL
0,BR1S00001,BR1,Sales,Terraced,"Palace View, Bromley, BR1",3,1,"£745,000",A fantastic opportunity to purchase this mid t...,Added on 27/03/2023,Allow all cookies,RightMove,https://www.rightmove.co.uk/property-for-sale/...
1,BR1S00002,BR1,Sales,Detached,"Provident House, 122-126 High Street, Bromley BR1",6,7,"£5,750,000",Extremely rare opportunity to purchase this tr...,NaN,Added on 27/03/2023 by Truepenny's Property Co...,RightMove,https://www.rightmove.co.uk/property-for-sale/...
2,BR1S00003,BR1,Sales,Detached,"Woodlands Close, Bromley",5,5,"£3,500,000","Guide Price £2,500,000 to £2,600,000. Brand ne...",Added on 25/03/2023,"Marketed by Knight Frank Land, London",RightMove,https://www.rightmove.co.uk/property-for-sale/...
3,BR1S00004,BR1,Sales,Detached,Sundridge Avenue Bromley BR1,6,3,"£2,500,000",A fantastic family home offered in quiet sough...,Added on 16/03/2023,"Added on 25/03/2023 by Alan de Maid, Chislehurst",RightMove,https://www.rightmove.co.uk/property-for-sale/...
4,BR1S00005,BR1,Sales,House,"Mavelstone Close, Bromley, Kent, BR1",8,3,"£2,400,000","Of interest to property developers, speculator...",Added on 22/02/2023,"Added on 16/03/2023 by Unique, Langford Russel...",RightMove,https://www.rightmove.co.uk/property-for-sale/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
49636,WC2RS00047,WC2R,Sales,Flat,"Clement House, 190 Strand, Covent Garden, Lond...",1,1,"£1,075,000",A rare opportunity to purchase a studio flat i...,Added on 17/11/2020,ONLINE VIEWING,RightMove,https://www.rightmove.co.uk/property-for-sale/...
49637,WC2RS00048,WC2R,Sales,Flat,"Charles Apartments, 1 Bull Inn Court, Covent G...",1,1,"£1,075,000",A rare opportunity to purchase a one bedroom f...,ONLINE VIEWING,"Added on 19/11/2020 by Knight Frank, Mayfair",RightMove,https://www.rightmove.co.uk/property-for-sale/...
49638,WC2RS00049,WC2R,Sales,Apartment,"Strand, Covent Garden, WC2R",1,1,"£1,500,000","A stunning one bedroom, one bathroom apartment...",Added on 17/11/2020,Allow all cookies,RightMove,https://www.rightmove.co.uk/property-for-sale/...
49639,WC2RS00050,WC2R,Sales,Apartment,"Wren House, 190 The Strand, London",1,1,"£900,000","This incredibly finished apartment, set in a q...",Reduced on 10/07/2020,"Added on 17/11/2020 by CBRE Residential, West End",RightMove,https://www.rightmove.co.uk/property-for-sale/...


<h1 style='background-color: #0B3861; padding: 10px; color: white'> RENT </h1>


Now, we will create another function called 'rightMove_rent' to scrap properties that are for rent on the Rightmove website

In [52]:
def rightMove_rent(postcodes, Trans_type, website, df):
    """
    This function scrapes property data from Rightmove for a given list of postcodes and transaction type,
    and returns the data as a pandas DataFrame.

    Arguments:
    postcodes -- a list of postcodes for which to scrape property data
    Trans_type -- the transaction type of the properties to be scraped ('sales' or 'rent')
    website -- the name of the website being scraped (in this case, 'Rightmove')
    df -- an empty pandas DataFrame to store the scraped data

    Returns:
    df1 -- a pandas DataFrame containing the scraped property data
    """
    
    # Set the path to the ChromeDriver executable
    s = Service("C:\\Users\\user\\Downloads\\Set ups\\chromedriver_win32\\chromedriver.exe")
    
    # Launch the ChromeDriver with the specified service
    driver = webdriver.Chrome(service= s)
    
    # Navigate to the specified URL and maximize the window
    driver.get('https://www.rightmove.co.uk/')
    driver.maximize_window()
    
    # Wait for 3 seconds to allow the page to load
    time.sleep(3)

    # Find the search bar and enter the search query
    time.sleep(2)
    search = driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/main/div[1]/div/div/div/div/input')
    search.send_keys(postcode) 
    time.sleep(2)
    
    #Clicking the find search button
    driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/main/div[1]/div/div/div/button[2]').click()
    time.sleep(2)
    
    #Clicking the find properties button
    driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div/div[1]/div/form/fieldset[2]/div[4]/button').click()
    
    # Initialize empty lists for storing scraped data
    Trans_type = []
    address = []
    types = []
    bedrooms = []
    bathrooms = []
    prices = []
    desc = []
    date_added = []
    agent_list = []
    property_url = []
    website = []
    
    # Create an empty dataframe to store the scraped data
    df = pd.DataFrame()
    
    # Setting the page number to be 1 and looping over the available pages for each postcode
    i = 1
    while True:
        
        # Scrapping data for the required features
        time.sleep(2)
        print("{} {} {} {}".format('scraping page', i,'from', postcode ))
        address_list = driver.find_elements(by='class name', value='propertyCard-address')
        type_list = driver.find_elements(By.XPATH, "//div/div/div[4]/div[1]/div[2]/a/div/span[1]")
        bedroom_list = driver.find_elements(By.XPATH, '//div/div/div[4]/div[1]/div[2]/a/div/span[3]')
        bathroom_list = driver.find_elements(By.XPATH, '//div/div/div[4]/div[1]/div[2]/a/div/span[5]')
        price_list = driver.find_elements(By.XPATH, '//div/div/div[3]/div/div/a/div/span[1]')
        desc_list = driver.find_elements(By.XPATH, '//div/div/div[4]/div[1]/div[3]/a/span/span')
        date_added_list = driver.find_elements(By.XPATH, '//div/div/div[4]/div[2]/div/span')
        agent_list_list = driver.find_elements(By.XPATH, '//div/div/div[4]/div[2]/div')
        property_url_list = driver.current_url
        Trans_type_list = Trans_type
        website_list = website
        for address_item, type_item, bedroom_item, bathroom_item, price_item, desc_item, date_added_item, agent_list_item in zip(address_list, type_list, bedroom_list, bathroom_list, price_list, desc_list, date_added_list, agent_list_list):
            address.append(address_item.text)
            types.append(type_item.text)
            bedrooms.append(bedroom_item.text)
            bathrooms.append(bathroom_item.text)
            prices.append(price_item.text)
            desc.append(desc_item.text)
            date_added.append(date_added_item.text)
            agent_list.append(agent_list_item.text)
            property_url.append(property_url_list)
            Trans_type.append(Trans_type_list)
            website.append(website_list)   
            
        # Find the next button   
        next_botton = driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[3]/div/div/div/div[3]/button/span')
        
        # Find the last next button
        last_botton_check = driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[3]/div/div/div/div[3]/button')
        
        # Checks for the last next button and breaks off
        html = last_botton_check.get_attribute('outerHTML') 
        if 'disabled' in html:
            print('------------------------------- SCRAPING COMPLETED FOR ' + postcode)
            break
            
        #Click the next button if it's not the last    
        next_botton.click()
        
        #Increment the page number
        i += 1
        time.sleep(2)
        
    # Create a dataframe to store data scrapped for each postcode
    df1 = pd.DataFrame({'Location': postcode, 'Tansaction_Type': 'Rent','Property_Type':types, 'Address' :address, 'Bedrooms': bedrooms, 'Bathrooms':bathrooms, 'Price':prices, 'Description': desc, 'Listing_Date':date_added, 'Agent':agent_list, 'Listing_Source': 'RightMove', 'listing_URL':property_url})
    df1.insert(0, 'Unique_Id', [f'{postcode}R{i+1:05d}' for i in range(len(df1))])
    print('Total numbers of properties available in ' + postcode + ' is ' + str(df1.shape[0]))

    #Concat the dataframe obtain for all postcodes
    df = pd.concat([df, df1], ignore_index=True)
    
    # Returning a dataframe
    return df1



Now, the data for properties available for rent at Rightmove is scrapped

In [53]:
# create an empty DataFrame outside the function
df = pd.DataFrame()

# loop through postcodes
for postcode in codes['Postcode district']:
    # call the function and pass the empty DataFrame as an argument
    df1 = rightMove_rent(postcode, 'Rent', 'RightMove', df)
    # append the df1 DataFrame to the empty DataFrame
    df = pd.concat([df, df1], ignore_index=True)
print('-------------------------------    ------------------------------- WEB SCRAPING COMPLETED')

scraping page 1 from BR1
scraping page 2 from BR1
scraping page 3 from BR1
scraping page 4 from BR1
------------------------------- SCRAPING COMPLETED FOR BR1
Total numbers of properties available in BR1 is 89
scraping page 1 from BR2
scraping page 2 from BR2
------------------------------- SCRAPING COMPLETED FOR BR2
Total numbers of properties available in BR2 is 28
scraping page 1 from BR3
scraping page 2 from BR3
------------------------------- SCRAPING COMPLETED FOR BR3
Total numbers of properties available in BR3 is 38
scraping page 1 from BR4
------------------------------- SCRAPING COMPLETED FOR BR4
Total numbers of properties available in BR4 is 7
scraping page 1 from BR5
------------------------------- SCRAPING COMPLETED FOR BR5
Total numbers of properties available in BR5 is 17
scraping page 1 from BR6
------------------------------- SCRAPING COMPLETED FOR BR6
Total numbers of properties available in BR6 is 18
scraping page 1 from BR7
------------------------------- SCRAPING 

KeyboardInterrupt: 

In case the web scraping breaks while running due to network, we can continue the scrapping process from the last postcode as follows

In [9]:
# create an empty DataFrame outside the function
df = pd.DataFrame()

# loop through postcodes
for postcode in codes.loc[253:]['Postcode district']:
    # call the function and pass the empty DataFrame as an argument
    df1 = rightMove_rent(postcode, 'Rent', 'RightMove', df)
    # append the df1 DataFrame to the empty DataFrame
    df = pd.concat([df, df1], ignore_index=True)
print('-------------------------------    ------------------------------- WEB SCRAPING COMPLETED')

scraping page 1 from W8
scraping page 2 from W8
scraping page 3 from W8
scraping page 4 from W8
scraping page 5 from W8
scraping page 6 from W8
scraping page 7 from W8
scraping page 8 from W8
scraping page 9 from W8
scraping page 10 from W8
scraping page 11 from W8
scraping page 12 from W8
scraping page 13 from W8
scraping page 14 from W8
scraping page 15 from W8
scraping page 16 from W8
scraping page 17 from W8
scraping page 18 from W8
scraping page 19 from W8
scraping page 20 from W8
scraping page 21 from W8
scraping page 22 from W8
scraping page 23 from W8
scraping page 24 from W8
------------------------------- SCRAPING COMPLETED FOR W8
(491, 13)
scraping page 1 from W9
scraping page 2 from W9
scraping page 3 from W9
scraping page 4 from W9
scraping page 5 from W9
scraping page 6 from W9
scraping page 7 from W9
scraping page 8 from W9
scraping page 9 from W9
scraping page 10 from W9
scraping page 11 from W9
------------------------------- SCRAPING COMPLETED FOR W9
(219, 13)
scrapin

Saving the last dataset obtained for rent at rightmove

In [10]:
df.to_csv('rightmove_rent_data.csv')

Loading the data for properties for rent at Rightmove

In [18]:
df_RM_rent = pd.read_csv('rightmove_rent_data.csv')

# Remove an unwanted column
df_RM_rent = df_RM_rent.drop('Unnamed: 0', axis=1)

In [19]:
# Check the shape of the dataset
df_RM_rent.shape

(21126, 13)

Over 21,000 properties was scrapped

In [20]:
df_RM_rent

,Unique_Id,Location,Tansaction_Type,Property_Type,Address,Bedrooms,Bathrooms,Price,Description,Listing_Date,Agent,Listing_Source,listing_URL
0,BR1R00001,BR1,Rent,Terraced,"Hillcrest Road, Bromley, BR1",3,1,"£2,000 pcm",We are delighted to present to the market this...,Reduced on 04/05/2023,Reduced on 04/05/2023 by Daniels Property Serv...,RightMove,https://www.rightmove.co.uk/property-to-rent/f...
1,BR1R00002,BR1,Rent,End of Terrace,Pendragon Road Bromley BR1,5,2,"£2,500 pcm",We are delighted to present this wonderfully s...,Reduced today,"Reduced today by Acorn, Grove Park",RightMove,https://www.rightmove.co.uk/property-to-rent/f...
2,BR1R00003,BR1,Rent,Flat,"Hillside, Bickley, BR1",2,1,"£1,500 pcm",*ZERO DEPOSIT OPTION AVAILABLE* *AVAILABLE NOW...,Reduced today,"Reduced today by Andrew Reeves, Bromley-Lettings",RightMove,https://www.rightmove.co.uk/property-to-rent/f...
3,BR1R00004,BR1,Rent,Studio,"**ALL BILLS INCLUDED** Coniston Road, Bromley,...",1,1,"£1,200 pcm","**ALL BILLS INCLUDED** - BEAUTIFUL, LARGE, MOD...",Reduced today,"Reduced today by Andrew Reeves, Bromley-Lettings",RightMove,https://www.rightmove.co.uk/property-to-rent/f...
4,BR1R00005,BR1,Rent,Flat,"Dale Court, Bromley, BR1",3,1,"£1,600 pcm","We are pleased to market this 3 bedroom, 1 bat...",Added on 07/05/2023,"Added on 07/05/2023 by OpenRent, London",RightMove,https://www.rightmove.co.uk/property-to-rent/f...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21121,WC2RR00017,WC2R,Rent,Apartment,"Gladstone House, 190 Strand London WC2R",3,3,"£8,450 pcm",Luxury three bedroom apartment in Gladstone Ho...,Added on 01/04/2023,"Added on 01/04/2023 by Chase Evans, City & Ald...",RightMove,https://www.rightmove.co.uk/property-to-rent/f...
21122,WC2RR00018,WC2R,Rent,Flat,"Clement House, 190 Strand, Westminster, London...",1,3,"£3,445 pcm",Elegant studio apartment situated on the third...,Added on 29/03/2023,Added on 29/03/2023 by Knight Frank - Lettings...,RightMove,https://www.rightmove.co.uk/property-to-rent/f...
21123,WC2RR00019,WC2R,Rent,Apartment,"190 The Strand, Temple House, WC2R",2,2,"£5,300 pcm",Regent are proud to present this spectacular t...,Reduced on 26/04/2023,Reduced on 26/04/2023 by Regent Letting and Pr...,RightMove,https://www.rightmove.co.uk/property-to-rent/f...
21124,WC2RR00020,WC2R,Rent,Apartment,"Essex Street, Temple, London, WC2R",3,3,"£12,567 pcm",United Kingdom Sotheby’s International Realty ...,Added on 14/09/2022,Added on 14/09/2022 by Sotheby's International...,RightMove,https://www.rightmove.co.uk/property-to-rent/f...
